In [4]:

import numpy as np


# sigmoid function


##### ease of differentiation and probability interpretation We are using the sigmoid function to convert our sign distances which are in any range (-infinity , +infinity) to [0,1].These transformed distances are nothing but the probability scores for each point no each side of the plane. The all -ve and +ve points which are correctly classified will have probabilities very high and if any outlier is present it will be having very low probability score

In [ ]:
#numpy version of sigmoid function
def sigmoid(z: float) -> float:
	#Your code here
    result = np.round(1 / (1 + np.exp(-z)),4)
    return result

In [11]:
print(sigmoid(0))
print(sigmoid(-1))
print(sigmoid(1))

0.5
0.2689
0.7311


In [14]:
# pytorch version of sigmoid function
def sigmoid(z: torch.Tensor) -> torch.Tensor:
    # result = 1 / (1 + torch.exp(-z))
    return torch.round(torch.sigmoid(z), decimals=4)

In [15]:
print(sigmoid(torch.tensor(0)).item())

0.5


# single neuron network and mean squared error

In [ ]:
def get_mse(predictions: np.ndarray, labels: np.ndarray) -> float: 

    error = predictions - labels
    squared_error = np.square(error)
    mse_value = np.mean(squared_error)
    return round(float(mse_value),4)

In [36]:
def single_neuron_model(features: list[list[float]], labels: list[int], weights: list[float], bias: float) -> (list[float], float):
	features_1 = np.array(features)  # Ensure features are a 2D array
	weights_1 = np.array(weights).reshape(-1, 1)  # Ensure weights are a column vecto
	labels = np.array(labels).reshape(-1, 1)  # Ensure labels are a column vector     
	dot_product = np.dot(features_1, weights_1)
	output = dot_product+bias
	probabilities = sigmoid(output)
	mse = get_mse(probabilities, labels)
	
	return probabilities, mse

In [37]:
print(single_neuron_model([[0.5, 1.0], [-1.5, -2.0], [2.0, 1.5]], [0, 1, 0], [0.7, -0.4], -0.1)) 
print(single_neuron_model([[1, 2], [2, 3], [3, 1]], [1, 0, 1], [0.5, -0.2], 0))

(array([[0.4626],
       [0.4134],
       [0.6682]]), 0.3349)
(array([[0.525 ],
       [0.5987],
       [0.7858]]), 0.21)


# Single Neuron with Backpropagation

### <img src="../img/img_3.png" width="900" height="500">
### <img src="../img/img_4.png" width="900" height="900">

In [6]:
import numpy as np

def sigmoid(z: np.ndarray) -> np.ndarray:
    result = 1 / (1 + np.exp(-z))
    return result

def sigmoid_derivative(x: np.ndarray) -> np.ndarray:
    # Sigmoid derivative: f'(z) = f(z) * (1 - f(z))
    # where f(z) is the sigmoid function, 
    # here x is the output of the sigmoid function
    return x*(1-x)

def forward_propogation(features: np.ndarray, bias: float, weights: np.ndarray) -> np.ndarray:
    output = np.dot(features, weights) + bias
    probabilities = sigmoid(output)
    return probabilities

def get_mse(predictions: np.ndarray, labels: np.ndarray) -> float: 
    mse_value = np.mean((predictions - labels) ** 2)
    return np.round(float(mse_value),4)

def mse_derivative(predictions: np.ndarray, labels: np.ndarray) -> np.ndarray:
    n = predictions.shape[0]
    return (2 / n) * (predictions - labels)

def back_propogation(predictions, labels, weight, bias, learning_rate, features) -> np.ndarray:
    # dL/dw = dL/dy' * dy'/dz * dz/dw (see above picture)
    error = mse_derivative(predictions, labels) #dL/y'
    delta = error * sigmoid_derivative(predictions) #dL/dy' * dy'/dz
    weights_output = weight - (np.dot(features.T, delta) * learning_rate) # w - (dL/dy * dy/dz * x)  we get x from dz/w i.e z = w*x + b
    bias_output = bias - (np.sum(delta) * learning_rate)
    return weights_output, bias_output


def train_neuron(features: np.ndarray, labels: np.ndarray, initial_weights: np.ndarray, initial_bias: float, learning_rate: float, epochs: int) -> (np.ndarray, float, list[float]):
    labels = np.array(labels)
    weights = np.array(initial_weights)
    bias = initial_bias
    mse_values = []

    for _ in range(epochs):
        # forward feed
        predictions = forward_propogation(features, bias, weights)
        mse_values.append(get_mse(predictions, labels))
        
        weights, bias = back_propogation(predictions, labels, weights, bias, learning_rate, features)
        weights = np.round(weights, 4)
        bias = round(bias, 4)
    
    updated_weights = weights
    updated_bias = bias
    
    return (
    updated_weights.tolist(),             
    round(float(updated_bias), 4),             
    np.round(mse_values, 4).tolist()           
    )

In [7]:
print(train_neuron(np.array([[1.0, 2.0], [2.0, 1.0], [-1.0, -2.0]]), np.array([1, 0, 0]), np.array([0.1, -0.2]), 0.0, 0.1, 2))
print(train_neuron(np.array([[1, 2], [2, 3], [3, 1]]), np.array([1, 0, 1]), np.array([0.5, -0.2]), 0, 0.1, 3))

([0.1036, -0.1426], -0.0167, [0.3033, 0.2942])
([0.4892, -0.2301], 0.0029, [0.21, 0.2087, 0.2076])
